In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras_tqdm import TQDMNotebookCallback
import keras.callbacks

from sqlalchemy import create_engine
import json
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
engine = create_engine('sqlite:///tab.db')
conn = engine.connect()
conn
dfo = pd.read_sql_table('race', conn)
print('{} races loaded!'.format(len(dfo)))

9587 races loaded!


In [3]:
category = 'G'
dfo = dfo.loc[dfo['race_type'].isin([category])]
print('{} {} races!'.format(len(dfo), category))

3931 G races!


In [4]:
dfo['results'] = dfo['results_data'].map(json.loads)
dfo['runners'] = dfo['runners_data'].map(json.loads)
dfo.head()

,id,meeting_name,location,venue_mnemonic,race_type,meeting_date,race_number,race_name,race_start_time,race_status,race_distance,results_data,num_runners,runners_data,results,runners
0,1,RICHMOND,NSW,C,G,2017-09-13,1,LADBROKES INFO HUB MDN HT1,2017-09-13 05:12:00,Paying,400,"[[4], [2], [1], [5]]",8.0,"[{""runnerName"": ""ZALI'S BLUEBOY"", ""runnerNumbe...","[[4], [2], [1], [5]]","[{'runnerName': 'ZALI'S BLUEBOY', 'runnerNumbe..."
1,2,RICHMOND,NSW,C,G,2017-09-13,2,KEVIN WATERS TOWING MDN HT2,2017-09-13 05:32:00,Paying,400,"[[1], [8], [7], [2]]",8.0,"[{""runnerName"": ""SKETCHY LINDA"", ""runnerNumber...","[[1], [8], [7], [2]]","[{'runnerName': 'SKETCHY LINDA', 'runnerNumber..."
2,3,RICHMOND,NSW,C,G,2017-09-13,4,PENRITHELECTRICAL.COM.AU STKS,2017-09-13 06:07:00,Paying,400,"[[9], [7], [8], [2]]",8.0,"[{""runnerName"": ""BROWZA BOB'"", ""runnerNumber"":...","[[9], [7], [8], [2]]","[{'runnerName': 'BROWZA BOB'', 'runnerNumber':..."
3,4,RICHMOND,NSW,C,G,2017-09-13,5,RICHMONDGREYHOUNDS.COM.AU STKS,2017-09-13 06:27:00,Paying,618,"[[6], [7], [3], [1]]",8.0,"[{""runnerName"": ""MIAMI DEMON"", ""runnerNumber"":...","[[6], [7], [3], [1]]","[{'runnerName': 'MIAMI DEMON', 'runnerNumber':..."
4,5,RICHMOND,NSW,C,G,2017-09-13,6,SWIFT FANCY @ STUD 1-2W STKS,2017-09-13 06:42:00,Paying,535,"[[6], [7], [1], [8]]",8.0,"[{""runnerName"": ""KILLING TIME"", ""runnerNumber""...","[[6], [7], [1], [8]]","[{'runnerName': 'KILLING TIME', 'runnerNumber'..."


In [5]:
# extract runners from races
data_all = pd.DataFrame()
for ri, race in dfo.iterrows():
#     print(race)

    num_runners = len([r for r in race['runners'] if r['odds_win']])

    # add num_runners to all runners
    for runner in race['runners']:
#         print(runner)
        runner['num_runners'] = num_runners
        runner['race_type'] = race['race_type']

    data_all = data_all.append(race['runners'])
    
data_all.tail(10)

,barrierNumber,claimAmount,finishingPosition,fixedOdds,num_runners,odds_perc,odds_scale,odds_win,parimutuel,prediction,probability,race_type,rank_win,riderDriverName,runnerName,runnerNumber,trainerName
2,3,0,0,"{'returnWin': 3.5, 'returnWinOpen': 1.01, 'ret...",6,0.285714,0.222677,3.5,"{'bettingStatus': 'Normal', 'returnWin': 5.5, ...",0.224591,0.213640,G,2,,TREBLE CHANCE,3,N F CARTER
3,4,0,1,"{'returnWin': 4.4, 'returnWinOpen': 1.01, 'ret...",6,0.227273,0.177129,4.4,"{'bettingStatus': 'Normal', 'returnWin': 4.7, ...",0.193449,0.184017,G,3,,TULLYMURRY BRUCE,4,J J LUCKHURST
4,5,0,3,"{'returnWin': 14, 'returnWinOpen': 1.01, 'retu...",6,0.071429,0.055669,14.0,"{'bettingStatus': 'Normal', 'returnWin': 12.7,...",0.052223,0.049677,G,6,,ROUGH ANNIE,5,A KELLY-PILGRIM
5,6,0,2,"{'returnWin': 5, 'returnWinOpen': 1.01, 'retur...",6,0.200000,0.155874,5.0,"{'bettingStatus': 'Normal', 'returnWin': 3.9, ...",0.167702,0.159525,G,4,,TWOOFAKIND,6,T M LEVERS
0,1,0,2,"{'returnWin': 6.5, 'returnWinOpen': 1.01, 'ret...",6,0.153846,0.110421,6.5,"{'bettingStatus': 'Normal', 'returnWin': 5.8, ...",0.117027,0.112354,G,5,,ISLE OF GOLD,1,A C B GREEN
1,2,0,1,"{'returnWin': 4, 'returnWinOpen': 1.01, 'retur...",6,0.250000,0.179435,4.0,"{'bettingStatus': 'Normal', 'returnWin': 4.4, ...",0.215846,0.207228,G,3,,EMMERDALE BELLE,2,T M LEVERS
2,3,0,0,"{'returnWin': 2.7, 'returnWinOpen': 1.01, 'ret...",6,0.370370,0.265829,2.7,"{'bettingStatus': 'Normal', 'returnWin': 6.9, ...",0.295085,0.283303,G,1,,ZATOPEKS BABY,3,G L DAVIDSON
3,4,0,0,"{'returnWin': 7.5, 'returnWinOpen': 1.01, 'ret...",6,0.133333,0.095699,7.5,"{'bettingStatus': 'Normal', 'returnWin': 5.5, ...",0.029391,0.028218,G,6,,SLIMTHETIC,4,G L DAVIDSON
4,5,0,3,"{'returnWin': 5, 'returnWinOpen': 1.01, 'retur...",6,0.200000,0.143548,5.0,"{'bettingStatus': 'Normal', 'returnWin': 5.8, ...",0.161861,0.155398,G,4,,THE OTHER JASMIN,5,I P STEVENS
5,6,0,0,"{'returnWin': 3.5, 'returnWinOpen': 1.01, 'ret...",6,0.285714,0.205068,3.5,"{'bettingStatus': 'Normal', 'returnWin': 3.5, ...",0.222379,0.213500,G,2,,MISFIT CHICA,6,J DALY


In [ ]:
# drop scratched
data = data_all.dropna(subset=['odds_win'])
data = data[(data.odds_win > 0)]
data.describe()

In [ ]:
# get label data
Y = data['finishingPosition'] == 1
Y = Y.astype(int)
Y.describe()

In [ ]:
# xp = fixed odds perc
xp = data['odds_perc']
xp.describe()

In [ ]:
# xs = odds scaled
xs = data['odds_scale']
xs.describe()

In [ ]:
# xr = win rank
xr = data['rank_win']
xr.describe()

In [ ]:
# xn = get num runners
xn = data['num_runners']
xn.describe()

In [ ]:
X = pd.concat([xp, xs, xr, xn], axis=1)
X.describe()
# sns.pairplot(pd.concat([X, Y], axis=1))

In [ ]:
# For a single-input model with 2 classes (binary classification):

n = len(X.columns)
print('input dimension = {}'.format(n))

epochs = 1000
print('epochs = {}'.format(epochs))

tag = '{}30x30'.format(category)
file_name = '/Users/jaco/code/tabby/models/{}.h5'.format(tag)

try:
    model = load_model(file_name)
except OSError:
    model = Sequential()
    model.add(Dense(30, activation='relu', input_dim=n))
    model.add(Dense(30, activation='relu'))
#     model.add(Dense(40, activation='relu'))
#     model.add(Dense(40, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [ ]:
# Train the model, iterating on the data in batches of 32 samples
tqdm = TQDMNotebookCallback()

tbCallBack = keras.callbacks.TensorBoard(
    log_dir='/Users/jaco/code/tabby/summary/{}'.format(tag), 
    histogram_freq=0,
    write_graph=True,
    write_images=True)

model.fit(
    X.as_matrix(), 
    Y.as_matrix(),
    validation_split=0.2,
    shuffle=True,
    epochs=epochs,
    batch_size=32,
    verbose=0,
    callbacks=[tqdm, tbCallBack])

# creates a HDF5 file 'my_model.h5'
model.save(file_name)